In [0]:
import pandas as pd
import io
import glob

In [0]:
#wont work
check = pd.read_csv("/Volumes/workspace/default/sports/20240301-AllenField-1.csv")
#display(df)
csv_files = glob.glob("/Volumes/workspace/default/sports/*.csv")
print(csv_files)
#pwd

In [0]:
df_spark = spark.read.csv("/Volumes/workspace/default/sports/*.csv", inferSchema=True, header=True)

In [0]:
#df_spark.columns

In [0]:
df = df_spark.toPandas()

In [0]:
df = pd.read_csv("/Volumes/workspace/default/sports/20240301-AllenField-1.csv")

In [0]:
df[['Batter', 'Inning', 'Top/Bottom', 'Outs', 'Balls', 'Strikes', 'OutsOnPlay', 'PitchCall', 'PlayResult', 'RunsScored', 'KorBB']][:10]

In [0]:
#setting the initial values
df['b1'] = 0
df['b2'] = 0
df['b3'] = 0
df['b1Runner'] = None
df['b2Runner'] = None
df['b3Runner'] = None
df['Event'] = None

In [0]:
def resetRunners(df, ind):
    df.loc[ind, 'b1'] = 0
    df.loc[ind, 'b2'] = 0
    df.loc[ind, 'b3'] = 0
    df.loc[ind, 'b1Runner'] = None
    df.loc[ind, 'b2Runner'] = None
    df.loc[ind, 'b3Runner'] = None

def pushDown(df, ind):
    #copy from previous row
    df.loc[ind+1, 'b1'] = df.loc[ind, 'b1']
    df.loc[ind+1, 'b2'] = df.loc[ind, 'b2']
    df.loc[ind+1, 'b3'] = df.loc[ind, 'b3']
    df.loc[ind+1, 'b1Runner'] = df.loc[ind, 'b1Runner']
    df.loc[ind+1, 'b2Runner'] = df.loc[ind, 'b2Runner']
    df.loc[ind+1, 'b3Runner'] = df.loc[ind, 'b3Runner']

def singleForced(df, ind):
    #updates the next row alone
    df.loc[ind+1, 'b1'] = 1
    df.loc[ind+1, 'b1Runner'] = df.loc[ind, 'Batter']

    if df.loc[ind, 'b1'] == 1:
        df.loc[ind+1, 'b2'] = 1
        df.loc[ind+1, 'b2Runner'] = df.loc[ind, 'b1Runner']
                
        if df.loc[ind, 'b2'] == 1:
            df.loc[ind+1, 'b3'] = 1
            df.loc[ind+1, 'b3Runner'] = df.loc[ind, 'b2Runner']

            if df.loc[ind, 'b3'] == 1:
                print(df.loc[ind, 'b3Runner'], " scored at ", ind)

def single(df, ind, sacrifice=False):
    #updates the next row alone
    runs_scored = 0
    if not sacrifice:
        df.loc[ind+1, 'b1'] = 1
        df.loc[ind+1, 'b1Runner'] = df.loc[ind, 'Batter']

    if df.loc[ind, 'b1'] == 1:
        df.loc[ind+1, 'b2'] = 1
        df.loc[ind+1, 'b2Runner'] = df.loc[ind, 'b1Runner']
                
    if df.loc[ind, 'b2'] == 1:
        df.loc[ind+1, 'b3'] = 1
        df.loc[ind+1, 'b3Runner'] = df.loc[ind, 'b2Runner']

    if df.loc[ind, 'b3'] == 1:
        runs_scored += 1
        print(df.loc[ind, 'b3Runner'], " scored at ", ind)
    
    if runs_scored != df.loc[ind, 'RunsScored']:
        print("***corretions to be made at ", ind)
        print("***b# might have ran bases/ need to consider outs on play")
    
    if df.loc[ind, 'OutsOnPlay'] != 0:
        print(df.loc[ind, 'OutsOnPlay'], " base runners are out at ", ind)

def double(df, ind):
    runs_scored = 0
    df.loc[ind+1, 'b2'] = 1
    df.loc[ind+1, 'b2Runner'] = df.loc[ind, 'Batter']

    if df.loc[ind, 'b1'] == 1:
        df.loc[ind+1, 'b3'] = 1
        df.loc[ind+1, 'b3Runner'] = df.loc[ind, 'b1Runner']

    if df.loc[ind, 'b2'] == 1:
        print(df.loc[ind, 'b2Runner'], " scored at ", ind)
        runs_scored += 1

    if df.loc[ind, 'b3'] == 1:
        print(df.loc[ind, 'b3Runner'], " scored at ", ind)
        runs_scored += 1
    
    if runs_scored != df.loc[ind, 'RunsScored']:
        print("***corretions to be made at ", ind)
        print("***b1 might have ran 3 bases/ need to consider outs on play")

    if df.loc[ind, 'OutsOnPlay'] != 0:
        print("***", df.loc[ind, 'OutsOnPlay'], " base runner are out at ", ind)


def triple(df, ind):
    runs_scored = 0
    df.loc[ind+1, 'b3'] = 1
    df.loc[ind+1, 'b3Runner'] = df.loc[ind, 'Batter']

    if df.loc[ind, 'b1'] == 1:
        print(df.loc[ind, 'b1Runner'], " scored at ", ind)
        runs_scored += 1

    if df.loc[ind, 'b2'] == 1:
        print(df.loc[ind, 'b2Runner'], " scored at ", ind)
        runs_scored += 1

    if df.loc[ind, 'b3'] == 1:
        print(df.loc[ind, 'b3Runner'], " scored at ", ind)
        runs_scored += 1
    
    if runs_scored != df.loc[ind, 'RunsScored']:
        print("***corretions to be made at ", ind)
        print("***need to consider outs on play")

    if df.loc[ind, 'OutsOnPlay'] != 0:
        print("***", df.loc[ind, 'OutsOnPlay'], " base runner are out at ", ind)
            

In [0]:
#setting the initial values
df['b1'] = 0
df['b2'] = 0
df['b3'] = 0
df['b1Runner'] = None
df['b2Runner'] = None
df['b3Runner'] = None

for ind in df.index[1:]:
    #inning change
    #this can be removed if I dont update every row when base runner proceeds
    #if this case no update
    if df.loc[ind, 'Top/Bottom'] != df.loc[ind-1, 'Top/Bottom'] \
    or df.loc[ind-1, 'Inning'] != df.loc[ind, 'Inning']:
        print('\tInning change at', ind)
        resetRunners(df, ind)

    #individual cases of runner moving
    else:
        #walk
        if df.loc[ind, 'KorBB'] == 'Walk':
            print("walk at ", ind)
            df.loc[ind, 'Event'] = 'Walk'
            singleForced(df, ind)
        #HBP
        elif df.loc[ind, 'PitchCall'] == 'HitByPitch':
            print("HBP at ", ind)
            df.loc[ind, 'Event'] = 'HBP'
            singleForced(df, ind)
        #single
        elif df.loc[ind, 'PlayResult'] == 'Single':
            print("single at ", ind)
            df.loc[ind, 'Event'] = 'Single'
            single(df, ind)
        elif df.loc[ind, 'PlayResult'] == 'Sacrifice':
            print("Sacrifice at ", ind)
            df.loc[ind, 'Event'] = 'Sacrifice'
            single(df, ind, sacrifice=True)

        #double
        elif df.loc[ind, 'PlayResult'] == 'Double':
            print("double at ", ind)
            df.loc[ind, 'Event'] = 'Double'
            double(df, ind)

        #triple
        elif df.loc[ind, 'PlayResult'] == 'Triple':
            print("triple at ", ind)
            df.loc[ind, 'Event'] = 'Triple'
            triple(df, ind)
        
        #else
        else:
            pushDown(df, ind)

In [0]:
df[['Batter', 'b1', 'b2', 'b3', 'b1Runner', 'b2Runner', 'b3Runner', 'OutsOnPlay', 'Event', 'RunsScored', 'KorBB']][10:60]